In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import sys
import platform
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# # required to enable non-blocking interaction:
# %gui qt5 ## TODO 2024-01-18 - this causes kernel to crash when running notebook remotely via VSCode's ssh remote

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget
from pyphocorehelpers.gui.Jupyter.simple_widgets import code_block_widget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass


from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import build_vscode_workspace, build_windows_powershell_run_script

from pyphocorehelpers.exception_helpers import CapturedException
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult, BatchSessionCompletionHandler

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, filesystem_path_folder_contents_widget

import inspect
from jinja2 import Template
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import MAIN_get_template_string, write_test_script
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import export_session_h5_file_completion_function, curr_runtime_context_header_template, export_rank_order_results_completion_function, figures_rank_order_results_completion_function, compute_and_export_marginals_dfs_completion_function, determine_session_t_delta_completion_function, perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, reload_exported_kdiba_session_position_info_mat_completion_function, compute_and_export_session_wcorr_shuffles_completion_function, compute_and_export_session_instantaneous_spike_rates_completion_function, compute_and_export_session_extended_placefield_peak_information_completion_function, compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function, backup_previous_session_files_completion_function, compute_and_export_session_trial_by_trial_performance_completion_function
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import ProcessingScriptPhases

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_cell_remapping_category


BATCH_DAY_DATE: str = '2024-10-23'
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_Apogee' # used for filenames throught the notebook
BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_GL' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_Lab' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_rMBP' # used for filenames throught the notebook

# scripts_output_path = Path('/home/halechr/cloud/turbo/Data/Output/gen_scripts/').resolve() # Greatlakes
# scripts_output_path = Path('output/gen_scripts/').resolve() # Apogee
# # scripts_output_path = Path('/home/halechr/FastData/gen_scripts/').resolve() # Lab
# assert scripts_output_path.exists()
known_scripts_output_paths = [Path(v).resolve() for v in ['/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/cloud/turbo/Data/Output/gen_scripts', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/']]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')

collected_outputs_path = scripts_output_path.joinpath('../collected_outputs').resolve()
collected_outputs_path.mkdir(exist_ok=True)
assert collected_outputs_path.exists()
print(f'collected_outputs_path: {collected_outputs_path}')

Automatic pdb calling has been turned OFF
scripts_output_path: /nfs/turbo/umms-kdiba/Pho/Output/gen_scripts
collected_outputs_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs


## Build Processing Scripts:

In [2]:
# Get the generated code for user-contributed functions:
phase_any_run_custom_user_completion_functions_dict = None
# phase_any_run_custom_user_completion_functions_dict = {
	# 'backup_previous_session_files_completion_function': backup_previous_session_files_completion_function,
    # "export_rank_order_results_completion_function": export_rank_order_results_completion_function, # ran 2024-09-26 3pm
# # #     # "figures_rank_order_results_completion_function": figures_rank_order_results_completion_function,
    # "compute_and_export_marginals_dfs_completion_function": compute_and_export_marginals_dfs_completion_function, # ran 2024-07-16 5am
#     "determine_session_t_delta_completion_function": determine_session_t_delta_completion_function,  # ran 2024-05-28 6am
    # 'perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function': perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, # ran 2024-09-26 3pm
    # 'compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function': compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, # ran 2024-09-26 3pm
    # 'reload_exported_kdiba_session_position_info_mat_completion_function': reload_exported_kdiba_session_position_info_mat_completion_function,
	# 'compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function': compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function, # ran 2024-07-16 5am 
# 	# 'compute_and_export_session_wcorr_shuffles_completion_function': compute_and_export_session_wcorr_shuffles_completion_function,
    # 'compute_and_export_session_instantaneous_spike_rates_completion_function': compute_and_export_session_instantaneous_spike_rates_completion_function,
    # 'compute_and_export_session_extended_placefield_peak_information_completion_function': compute_and_export_session_extended_placefield_peak_information_completion_function,
    # 'compute_and_export_session_trial_by_trial_performance_completion_function': compute_and_export_session_trial_by_trial_performance_completion_function, 
    # 'export_session_h5_file_completion_function': export_session_h5_file_completion_function, # ran 2024-09-26 3pm
# }

In [3]:
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import BatchScriptsCollection, generate_batch_single_session_scripts, display_generated_scripts_ipywidget

# Hardcoded included_session_contexts:
included_session_contexts: List[IdentifyingContext] = UserAnnotationsManager.get_hardcoded_good_sessions()


## Setup Functions:
# force_recompute_override_computations_includelist = ['rank_order_shuffle_analysis', '_add_extended_pf_peak_information',
#     '_build_trial_by_trial_activity_metrics',
#     '_decode_and_evaluate_epochs_using_directional_decoders',
#     '_decode_continuous_using_directional_decoders',
#     '_decoded_epochs_heuristic_scoring',
#     '_split_train_test_laps_data',
#     'perform_wcorr_shuffle_analysis'
# ]

force_recompute_override_computations_includelist = None
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs',]
# force_recompute_override_computations_includelist = ['split_to_directional_laps','merged_directional_placefields','directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['directional_decoders_evaluate_epochs','directional_decoders_epoch_heuristic_scoring']
# force_recompute_override_computations_includelist = ['lap_direction_determination', 'split_to_directional_laps', 'pf_computation', 'pfdt_computation','firing_rate_trends','merged_directional_placefields','rank_order_shuffle_analysis',]
# force_recompute_override_computations_includelist = ['firing_rate_trends', 'extended_stats', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', ]
# force_recompute_override_computations_includelist = ['rank_order_shuffle_analysis','wcorr_shuffle_analysis','trial_by_trial_metrics']

force_recompute_override_computation_kwargs_dict = None
# force_recompute_override_computation_kwargs_dict = {'rank_order_shuffle_analysis':({'num_shuffles': 500, 'skip_laps': False} | dict(minimum_inclusion_fr_Hz=2.0, included_qclu_values=[1,2,4,5,6,7]))}
# computation_kwargs_dict={'rank_order_shuffle_analysis':({'num_shuffles': 500, 'skip_laps': False} | dict(minimum_inclusion_fr_Hz=2.0, included_qclu_values=[1,2,4,5,6,7]))}


extra_extended_computations_include_includelist = [
	# 'wcorr_shuffle_analysis'
]

custom_phase_extended_computations_include_includelist = None
# custom_phase_extended_computations_include_includelist=['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends',
# 	'pf_dt_sequential_surprise',
# 	'extended_stats',
# 	'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 
# 	'ratemap_peaks_prominence2d',
# 	'long_short_inst_spike_rate_groups',
# 	'long_short_endcap_analysis',
# 	# 'spike_burst_detection',
# 	'split_to_directional_laps',
# 	'merged_directional_placefields',
# 	'rank_order_shuffle_analysis',
# 	# 'directional_train_test_split',
# 	'directional_decoders_decode_continuous',
# 	'directional_decoders_evaluate_epochs',
# 	'directional_decoders_epoch_heuristic_scoring',
#     'wcorr_shuffle_analysis',
#     'trial_by_trial_metrics',
#     'extended_pf_peak_information',
# ]

active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'

# debug_print = True
debug_print = False
known_global_data_root_parent_paths = [Path('/Users/pho/data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')] # Path(r'/home/halechr/cloud/turbo/Data'), , Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

## Different run configurations:

# active_phase = ProcessingScriptPhases.clean_run
# active_phase = ProcessingScriptPhases.continued_run
# active_phase = ProcessingScriptPhases.final_run
active_phase = ProcessingScriptPhases.figure_run

custom_user_completion_functions_dict = active_phase.get_custom_user_completion_functions_dict(extra_run_functions=phase_any_run_custom_user_completion_functions_dict)
# print(f'custom_user_completion_functions_dict: {custom_user_completion_functions_dict}')
custom_user_completion_function_template_code, custom_user_completion_functions_dict = MAIN_get_template_string(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, collected_outputs_path=collected_outputs_path, override_custom_user_completion_functions_dict=custom_user_completion_functions_dict)
# print(f'custom_user_completion_function_template_code: {custom_user_completion_function_template_code}')
active_phase_dict = active_phase.get_run_configuration(custom_user_completion_function_template_code=custom_user_completion_function_template_code, extra_extended_computations_include_includelist=extra_extended_computations_include_includelist)

## Completely replace with custom functions:
if custom_phase_extended_computations_include_includelist is not None:
	print(f'WARNING: custom_phase_extended_computations_include_includelist: {custom_phase_extended_computations_include_includelist} is provided so only these extended_computation_fns will be used (overwritting the phase defaults!).')
	active_phase_dict['extended_computations_include_includelist'] = custom_phase_extended_computations_include_includelist

# active_phase_dict['extended_computations_include_includelist'].remove('wcorr_shuffle_analysis')
# active_phase_dict['should_freeze_pipeline_updates'] = True
active_phase_dict['should_create_vscode_workspace'] = True


## Build Slurm Scripts:
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
batch_scripts_collection: BatchScriptsCollection = generate_batch_single_session_scripts(global_data_root_parent_path, session_batch_basedirs=session_basedirs_dict, included_session_contexts=included_session_contexts,
                                                                                        output_directory=scripts_output_path, use_separate_run_directories=True,
                                                                                        # should_freeze_pipeline_updates=False, 
                                                                                        # create_slurm_scripts=True, 
                                                                                        # should_create_vscode_workspace=True,
                                                                                        # extended_computations_include_includelist=extended_computations_include_includelist,
                                                                                        force_recompute_override_computations_includelist=force_recompute_override_computations_includelist, # ['split_to_directional_laps', 'rank_order_shuffle_analysis'],
                                                                                        force_recompute_override_computation_kwargs_dict=force_recompute_override_computation_kwargs_dict,
																						# should_use_file_redirected_output_logging=True,
                                                                                        should_use_file_redirected_output_logging=True,
                                                                                        # should_perform_figure_generation_to_file=False,
                                                                                        # batch_session_completion_handler_kwargs=dict(enable_hdf5_output=False),
                                                                                        # custom_user_completion_functions=custom_user_completion_functions,
                                                                                        # custom_user_completion_function_template_code=custom_user_completion_function_template_code,
                                                                                        # should_force_reload_all=True,
                                                                                        # should_force_reload_all=False,
                                                                                        **active_phase_dict
                                                                                    )


# batch_scripts_collection.included_session_contexts, output_python_scripts, output_slurm_scripts

output_python_scripts = batch_scripts_collection.output_python_scripts
output_slurm_scripts = batch_scripts_collection.output_slurm_scripts
vscode_workspace_path = batch_scripts_collection.vscode_workspace_path
if vscode_workspace_path is not None:
    display(fullwidth_path_widget(vscode_workspace_path, file_name_label='vscode_workspace_path:'))

print(f"extended_computations_include_includelist: {active_phase_dict['extended_computations_include_includelist']}")
computation_script_paths = [x[0] for x in output_python_scripts]
generate_figures_script_paths = [x[1] for x in output_python_scripts]
print(f'generate_figures_script_paths: {generate_figures_script_paths}')
_out_widget = display_generated_scripts_ipywidget(batch_scripts_collection.included_session_contexts, output_python_scripts)
display(_out_widget)

script_folders: [PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/

Box(children=(Label(value='vscode_workspace_path:', layout=Layout(width='auto')), HTML(value="<b style='font-s…

extended_computations_include_includelist: ['lap_direction_determination', 'pf_computation', 'pfdt_computation', 'firing_rate_trends', 'pf_dt_sequential_surprise', 'extended_stats', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'ratemap_peaks_prominence2d', 'long_short_inst_spike_rate_groups', 'long_short_endcap_analysis', 'split_to_directional_laps', 'merged_directional_placefields', 'rank_order_shuffle_analysis', 'directional_train_test_split', 'directional_decoders_decode_continuous', 'directional_decoders_evaluate_epochs', 'directional_decoders_epoch_heuristic_scoring', 'extended_pf_peak_information']
generate_figures_script_paths: ['/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/figures_kdiba_gor01_one_2006-6-08_14-26-15.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/figures_kdiba_gor01_on

In [5]:
## INPUTS: active_phase, output_slurm_scripts, computation_script_paths, generate_figures_script_paths
also_show_figure_script_outputs: bool = True

# Maximum number of parallel script executions
max_parallel_executions = 1
# List of your script paths
# if active_phase.value == ProcessingScriptPhases.figure_run:
if active_phase.is_figure_phase:
    print(f'fig mode!')
    script_paths = generate_figures_script_paths
else:
	script_paths = computation_script_paths



if (platform.system() == 'Windows'):
	powershell_script_path = build_windows_powershell_run_script(script_paths, max_concurrent_jobs=max_parallel_executions, script_name='run_scripts')
	# print(f'powershell_script_path: {powershell_script_path}')
	display(fullwidth_path_widget(powershell_script_path, file_name_label='powershell_script_path:'))

	if also_show_figure_script_outputs or active_phase_dict['should_perform_figure_generation_to_file']:
		powershell_figures_script_path = build_windows_powershell_run_script(generate_figures_script_paths, max_concurrent_jobs=1, script_name='run_figure_scripts')
		display(fullwidth_path_widget(powershell_figures_script_path, file_name_label='powershell_figures_script_path:'))


if (platform.system() != 'Windows'):
	## Linux Only: Slurm Scripts
	# sbatch_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts])
	sbatch_run_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['run']])
	sbatch_figs_start_slurm_scripts: str = "\n".join([f"sbatch '{slurm_script}'" for slurm_script in output_slurm_scripts['figs']])

	if not active_phase.is_figure_phase:
		# Create and display the code block widget
		run_slurm_code_block = code_block_widget(sbatch_run_start_slurm_scripts, label="RunMain:")

	if (active_phase.is_figure_phase or also_show_figure_script_outputs and (len(sbatch_figs_start_slurm_scripts)>0)):
		# Create and display the code block widget
		figs_slurm_code_block = code_block_widget(sbatch_figs_start_slurm_scripts, label="Figures:")

# Batch 1
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44/run_kdiba_pin01_one_11-02_17-46-44.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0/run_kdiba_pin01_one_11-02_19-28-0.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/run_kdiba_pin01_one_11-03_12-3-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/run_kdiba_pin01_one_fet11-01_12-58-54.sh'

# Batch 2:
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/run_kdiba_vvp01_one_2006-4-09_17-29-30.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/run_kdiba_vvp01_one_2006-4-10_12-25-50.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/run_kdiba_vvp01_two_2006-4-09_16-40-54.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/run_kdiba_vvp01_two_2006-4-10_12-58-3.sh'

# Batch 3
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/run_kdiba_gor01_one_2006-6-08_14-26-15.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/run_kdiba_gor01_one_2006-6-09_1-22-43.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/run_kdiba_gor01_two_2006-6-07_16-40-19.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/run_kdiba_gor01_two_2006-6-08_21-16-25.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/run_kdiba_gor01_two_2006-6-09_22-24-40.sh'
# sbatch '/nfs/turbo/umms-kdiba/Data/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/run_kdiba_gor01_two_2006-6-12_16-53-46.sh'


HTML(value='<img src onerror="eval(navigator.clipboard.writeText(`sbatch \'/nfs/turbo/umms-kdiba/Pho/Output/ge…

## Execute the generated scripts in parallel:

In [ ]:
import subprocess
from concurrent.futures import ProcessPoolExecutor, as_completed
import ipywidgets as widgets
from IPython.display import display

## recieves: max_parallel_executions, script_paths
"""
# Maximum number of parallel script executions
max_parallel_executions = 5
# List of your script paths
script_paths = output_python_scripts
"""

# Function to execute a script
def run_script(script_path):
    python_executable = 'python'
    # python_executable = '/home/halechr/Library/VSCode/green/.venv_green/bin/python'
    
    try:
        result = subprocess.run([python_executable, script_path], capture_output=True, text=True)
        return script_path, result.stdout, result.stderr
    except BaseException as e:
        return script_path, None, str(e)

# Create a progress bar
progress_bar = widgets.IntProgress(value=0, min=0, max=len(script_paths), description='Running:', bar_style='info')
display(progress_bar)

# Run scripts in parallel with a limit on the number of parallel instances
with ProcessPoolExecutor(max_workers=max_parallel_executions) as executor:
    futures = {executor.submit(run_script, path): path for path in script_paths}
    for future in as_completed(futures):
        script, stdout, stderr = future.result()
        progress_bar.value += 1  # Update the progress bar
        if stderr:
            print(f"Error in {script}: {stderr}")
        else:
            print(f"Completed {script}")

# Progress bar will automatically update as scripts complete



## ⛓️🟢 Post-Processing of batch outputs:

In [6]:
from pyphocorehelpers.Filesystem.path_helpers import save_filelist_to_text_file


debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
## OUTPUTS: included_h5_paths, included_session_contexts, good_session_concrete_folders

## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

included_h5_paths = [Path(get_file_str_if_file_exists(v.pipeline_results_h5)).resolve() for v in good_session_concrete_folders]
check_output_h5_files(included_h5_paths)
included_h5_paths

def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
    """ Captures `BATCH_DATE_TO_USE` """
    # a_session_folder.context
    if session_context is not None:
        session_descr = session_context.session_name # '2006-6-07_16-40-19'
    _filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
    if len(args) > 0:
        _filename_list.extend([str(a_part) for a_part in args if a_part is not None])
    return separator_char.join(_filename_list)

copy_h5_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=collected_outputs_path, backup_mode=BackupMethods.CommonTargetDirectory, rename_backup_basename_fn=_across_session_h5_output_basename_fn, only_include_file_types=['h5']) # , rename_backup_suffix=BATCH_DATE_TO_USE
copy_h5_dict


## INPUTS: target_dir, BATCH_DATE_TO_USE
h5_filelist_output_filename=f'{BATCH_DATE_TO_USE}_all_sessions_h5_filelist.txt'
h5_filelist_output_file_path = Path(target_dir).joinpath(h5_filelist_output_filename).resolve() # Use Default
print(f'h5_filelist_output_file_path: {h5_filelist_output_file_path}')
_out_string, filelist_path = save_filelist_to_text_file(included_h5_paths, filelist_path=h5_filelist_output_file_path, debug_print=True) # r"W:\Data\all_sessions_h5_filelist_2024-03-28_Apogee.txt"
filelist_path

h5_filelist_output_file_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL_all_sessions_h5_filelist.txt
/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/vvp01/one/2006-4-09_17-29-30/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/vvp01/one/2006-4-10_12-25-50/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/vvp01/two/2006-4-09_16-40-54/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/pin01/one/11-03_12-3-25/output/pipeline_results.h5
/nfs/turbo/umms-kdiba/KDIBA/pin01/one/fet11

PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL_all_sessions_h5_filelist.txt')

# Output File Processing Helpers

## Copy exported `pipeline_results.h5` files into the `collected_outputs` folder, adding the current date
```
copying "W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\pipeline_results.h5"
		 -> "K:\scratch\collected_outputs\2024-06-12_Apogee_2006-6-08_14-26-15_pipeline_results.h5"...
```

In [7]:
a_copy_dict = copy_h5_dict

# Copies each file sequentially to the collected_outputs directory, and then builds an output file list
## INPUT a_copy_dict
moved_files_dict_h5_files = copy_movedict(a_copy_dict)
moved_files_dict_h5_files
# INPUTS: active_filelist_prefix, target_dir
# active_filelist_prefix: str = 'backed_up_files'
active_filelist_prefix: str = 'session_h5_files'

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

moved_files_copydict_output_filename=f'{active_filelist_prefix}_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = target_dir.joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: "{moved_files_copydict_file_path}"')

_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL_2006-6-08_14-26-15_pipeline_results.h5"...
done.
copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL_2006-6-09_1-22-43_pipeline_results.h5"...
done.
copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL_2006-6-12_15-55-31_pipeline_results.h5"...
done.
copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL_2006-6-07_16-40-19_pipeline_results.h5"...
done.
copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/collecte

## 2024-09-04 - Batch Output Files

In [9]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_session_inst_fr_data_to_across_session_pkl

## Complete Concise Run:
# RESULT_DATE_TO_USE = '2024-09-26'

RESULT_DATE_TO_USE: str = BATCH_DAY_DATE
print(f'RESULT_DATE_TO_USE: {RESULT_DATE_TO_USE}')

# instantaneous_time_bin_size_seconds_list = [0.002, 0.005, 0.025, 1000.0]
instantaneous_time_bin_size_seconds_list = [1000.0]

across_sessions_recomputed_instantaneous_fr_dict, moved_files_dict_files, (filelist_path, filedict_out_path) = copy_session_inst_fr_data_to_across_session_pkl(RESULT_DATE_TO_USE=RESULT_DATE_TO_USE, collected_outputs_path=collected_outputs_path, instantaneous_time_bin_size_seconds_list=instantaneous_time_bin_size_seconds_list)


# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

RESULT_DATE_TO_USE: 2024-10-22
custom_file_types_dict: {'recomputed_inst_fr_comps_1000.0': <function copy_session_inst_fr_data_to_across_session_pkl.<locals>.<dictcomp>.<lambda> at 0x149385bfdc10>}
moved_files_copydict_file_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22/recomputed_inst_fr_comps/2024-10-22_all_sessions_copydict_recomputed_inst_fr_comps_1000.0.csv
custom_filelist_output_file_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22/recomputed_inst_fr_comps/2024-10-22_all_sessions_recomputed_inst_fr_comps_1000.0_filelist.txt
PERFORMING THE MOVE....
copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-08_14-26-15/output/2024-10-22_recomputed_inst_fr_comps_1000.0.pkl"
		 -> "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22/recomputed_inst_fr_comps/2024-10-22_2006-6-08_14-26-15_2024-10-22_recomputed_inst_fr_comps_1000.0.pkl"...
done.
copying "/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-09_1-22-43/output/2024-10-22_recomputed_inst

# ! After Sessions are processed and their `.h5` files are exported, build the combined tables
The combined (across session) tables are saved out into .csv and .pkl formats: `W:\Data\2024-06-12_Apogee\neuron_identities_table.csv` -- ideally this would be in the combined outputs folder

In [10]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## FILE_OUTPUTS: '/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL'

## INPUTS: included_session_contexts, included_h5_paths, global_data_root_parent_path, BATCH_DATE_TO_USE
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True, )

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path
assert target_dir.exists(), f'target_dir: "{target_dir}" does not exist!'
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths, override_output_parent_path=target_dir, output_path_suffix=f'{BATCH_DATE_TO_USE}')
curr_collected_outputs_folder = Path(output_path_dicts['neuron_replay_stats_table']['.csv']).resolve().parent
filesystem_path_folder_contents_widget(curr_collected_outputs_folder)

failed for file path: /nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-08_14-26-15/neuron_identities/table. wth exception HDF5 error back trace

  File "H5F.c", line 836, in H5Fopen
    unable to synchronously open file
  File "H5F.c", line 796, in H5F__open_api_common
    unable to open file
  File "H5VLcallback.c", line 3863, in H5VL_file_open
    open failed
  File "H5VLcallback.c", line 3675, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 128, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 2003, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 600, in H5F__super_read
    truncated file: eof = 1170225306, sblock->base_addr = 0, stored_eof = 1170227549

End of HDF5 error back trace

Unable to open/create file '/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5'. Skipping.
failed for file path: /nfs/turbo/umms-kdiba/

In [11]:
neuron_replay_stats_table

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,...,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,neuron_uid,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,kdiba,gor01,one,2006-6-12_15-55-31,184.118896,NaN,LEFT_ONLY,1.219400,0.7021526081105963,-0.5172469579873612,...,147.835927,NaN,-7.638520,NaN,kdiba|gor01|one|2006-6-12_15-55-31,kdiba|gor01|one|2006-6-12_15-55-31|2,2006-06-12 15:55:31,8,4,False
1,kdiba,gor01,one,2006-6-12_15-55-31,100.095178,NaN,LEFT_ONLY,0.736015,0.5604191993558274,-0.1755954554186504,...,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,kdiba|gor01|one|2006-6-12_15-55-31|3,2006-06-12 15:55:31,8,4,False
2,kdiba,gor01,one,2006-6-12_15-55-31,157.384077,149.745557,SHARED,0.627405,0.3899675093283182,-0.23743737650351487,...,186.028526,197.486305,7.638520,141.312616,kdiba|gor01|one|2006-6-12_15-55-31,kdiba|gor01|one|2006-6-12_15-55-31|4,2006-06-12 15:55:31,8,4,False
3,kdiba,gor01,one,2006-6-12_15-55-31,180.299636,199.395935,SHARED,0.979698,1.6614524261148456,0.6817546561457393,...,82.908509,79.089249,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,kdiba|gor01|one|2006-6-12_15-55-31|5,2006-06-12 15:55:31,8,4,False
4,kdiba,gor01,one,2006-6-12_15-55-31,NaN,NaN,SHARED,10.865067,13.557340889454677,2.692273560204683,...,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,kdiba|gor01|one|2006-6-12_15-55-31|6,2006-06-12 15:55:31,8,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,1.3169564137781686,0.08943915035797345,...,130.255222,100.438316,74.542266,40.998246,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,2009-11-01 12:58:54,0,0,True
394,kdiba,pin01,one,fet11-01_12-58-54,57.576513,102.301872,SHARED,0.929991,0.8388929037677681,-0.0910981891459075,...,152.617902,126.528109,29.816906,11.181340,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29,2009-11-01 12:58:54,0,0,True
395,kdiba,pin01,one,fet11-01_12-58-54,120.937439,128.391665,SHARED,0.952187,0.8423839925259716,-0.10980286376952897,...,204.797488,186.161921,-18.635566,-52.179586,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|30,2009-11-01 12:58:54,0,0,True
396,kdiba,pin01,one,fet11-01_12-58-54,236.477951,NaN,LEFT_ONLY,1.156581,1.026152466402774,-0.13042850200623213,...,115.346769,96.711202,52.179586,0.000000,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|31,2009-11-01 12:58:54,0,0,True


# 2024-06-12 11:18: - [ ] HACK: reads back in the exported `neuron_replay_stats_table` CSV to fix an issue with pd.NA and np.nan not reading a NaNs and the dtypes of the columns being messed up on load:

In [13]:
_curr_read_path = output_path_dicts['neuron_replay_stats_table']['.csv']
# _curr_read_path = Path('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee/neuron_replay_stats_table.csv').resolve()
neuron_replay_stats_table: pd.DataFrame = pd.read_csv(_curr_read_path, na_values=['', 'nan', 'np.nan','<NA>'],
                                        dtype={
                                            'long_pf_peak_x': 'float64', 
                                            'short_pf_peak_x': 'float64',
                                            'long_LR_pf2D_peak_x': 'float64',
                                            'long_LR_pf2D_peak_y': 'float64',
                                            'long_RL_pf2D_peak_x': 'float64',
                                            'long_RL_pf2D_peak_y': 'float64',
                                            'short_LR_pf2D_peak_x': 'float64',
                                            'short_LR_pf2D_peak_y': 'float64',
                                            'short_RL_pf2D_peak_x': 'float64',
                                            'short_RL_pf2D_peak_y': 'float64',
                                            'long_LR_pf1D_peak': 'float64',
                                            'long_RL_pf1D_peak': 'float64',
                                            'short_LR_pf1D_peak': 'float64',
                                            'short_RL_pf1D_peak': 'float64',
                                            'peak_diff_RL_pf1D_peak': 'float64',
                                            'peak_diff_LR_pf1D_peak': 'float64'
                                        }, index_col='neuron_uid')
neuron_replay_stats_table


,Unnamed: 0,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,...,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
neuron_uid,,,,,,,,,,,,,,,,,,,,,
kdiba|gor01|one|2006-6-12_15-55-31|2,0,kdiba,gor01,one,2006-6-12_15-55-31,184.118896,NaN,LEFT_ONLY,1.219400,0.702153,...,NaN,147.835927,NaN,-7.638520,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False
kdiba|gor01|one|2006-6-12_15-55-31|3,1,kdiba,gor01,one,2006-6-12_15-55-31,100.095178,NaN,LEFT_ONLY,0.736015,0.560419,...,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False
kdiba|gor01|one|2006-6-12_15-55-31|4,2,kdiba,gor01,one,2006-6-12_15-55-31,157.384077,149.745557,SHARED,0.627405,0.389968,...,56.173689,186.028526,197.486305,7.638520,141.312616,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False
kdiba|gor01|one|2006-6-12_15-55-31|5,3,kdiba,gor01,one,2006-6-12_15-55-31,180.299636,199.395935,SHARED,0.979698,1.661452,...,NaN,82.908509,79.089249,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False
kdiba|gor01|one|2006-6-12_15-55-31|6,4,kdiba,gor01,one,2006-6-12_15-55-31,NaN,NaN,SHARED,10.865067,13.557341,...,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kdiba|pin01|one|fet11-01_12-58-54|28,393,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,1.316956,...,59.440069,130.255222,100.438316,74.542266,40.998246,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,0,0,True
kdiba|pin01|one|fet11-01_12-58-54|29,394,kdiba,pin01,one,fet11-01_12-58-54,57.576513,102.301872,SHARED,0.929991,0.838893,...,115.346769,152.617902,126.528109,29.816906,11.181340,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,0,0,True
kdiba|pin01|one|fet11-01_12-58-54|30,395,kdiba,pin01,one,fet11-01_12-58-54,120.937439,128.391665,SHARED,0.952187,0.842384,...,238.341507,204.797488,186.161921,-18.635566,-52.179586,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,0,0,True


In [14]:
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TruncationCheckingResults

# truncation_checking_aclus_dict, jonathan_firing_rate_analysis_result.neuron_replay_stats_df = truncation_checking_result.build_truncation_checking_aclus_dict(neuron_replay_stats_df=jonathan_firing_rate_analysis_result.neuron_replay_stats_df)

# 'is_rate_extrema', 'is_refined_exclusive', 'is_refined_LxC', 'is_refined_SxC'
# 'is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_cell_remapping_category

neuron_replay_stats_df = deepcopy(neuron_replay_stats_table)

neuron_replay_stats_df, (non_disappearing_endcap_cells_df, disappearing_endcap_cells_df, minorly_changed_endcap_cells_df, significant_distant_remapping_endcap_cells_df) = _add_cell_remapping_category(neuron_replay_stats_df=neuron_replay_stats_df,
                                                       loaded_track_limits = {'long_xlim': np.array([59.0774, 228.69]), 'short_xlim': np.array([94.0156, 193.757]), 'long_ylim': np.array([138.164, 146.12]), 'short_ylim': np.array([138.021, 146.263])},
)
# neuron_replay_stats_df
significant_distant_remapping_endcap_cells_df

loaded_track_limits: {'long_xlim': array([ 59.0774, 228.69  ]), 'short_xlim': array([ 94.0156, 193.757 ]), 'long_ylim': array([138.164, 146.12 ]), 'short_ylim': array([138.021, 146.263])}
num_disappearing_endcap_cells/num_total_endcap_cells: 5/69
num_non_disappearing_endcap_cells/num_total_endcap_cells: 64/69
num_significant_position_remappping_endcap_cells/num_non_disappearing_endcap_cells: 10/64


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/SpecificResults/PendingNotebookCode.py:1648: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_disappearing_endcap_cells_df['has_significant_distance_remapping'] = (np.abs(non_disappearing_endcap_cells_df['LS_pf_peak_x_diff']) >= min_significant_remapping_x_distance) # The most a placefield could translate intwards would be (35 + (pf_width/2.0)) I think.


,Unnamed: 0,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,...,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,has_long_pf,has_short_pf,has_significant_distance_remapping
neuron_uid,,,,,,,,,,,,,,,,,,,,,
kdiba|gor01|one|2006-6-12_15-55-31|32,30,kdiba,gor01,one,2006-6-12_15-55-31,249.046314,168.841856,SHARED,1.216096,2.123720,...,-15.277040,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,True,True
kdiba|gor01|one|2006-6-12_15-55-31|45,43,kdiba,gor01,one,2006-6-12_15-55-31,38.987020,92.456658,SHARED,4.865325,2.699642,...,NaN,-7.638520,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,True,True
kdiba|gor01|two|2006-6-07_16-40-19|64,114,kdiba,gor01,two,2006-6-07_16-40-19,235.301279,103.152020,SHARED,2.832245,5.618467,...,NaN,-15.102773,kdiba|gor01|two|2006-6-07_16-40-19,2006-06-07 16:40:19,1,0,True,True,True,True
kdiba|gor01|two|2006-6-12_16-53-46|24,138,kdiba,gor01,two,2006-6-12_16-53-46,54.226198,179.984124,SHARED,0.202350,2.313323,...,-99.082002,NaN,kdiba|gor01|two|2006-6-12_16-53-46,2006-06-12 16:53:46,9,4,False,True,True,True
kdiba|vvp01|one|2006-4-09_17-29-30|22,195,kdiba,vvp01,one,2006-4-09_17-29-30,34.270932,91.548868,SHARED,0.953844,0.869835,...,0.000000,NaN,kdiba|vvp01|one|2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,True,True,True,True
kdiba|vvp01|one|2006-4-09_17-29-30|42,215,kdiba,vvp01,one,2006-4-09_17-29-30,244.290031,164.100921,SHARED,1.102559,1.466096,...,NaN,NaN,kdiba|vvp01|one|2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,True,True,True,True
kdiba|vvp01|one|2006-4-09_17-29-30|43,216,kdiba,vvp01,one,2006-4-09_17-29-30,30.452403,87.730339,SHARED,1.953906,2.713736,...,NaN,3.818529,kdiba|vvp01|one|2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,True,True,True,True
kdiba|vvp01|one|2006-4-10_12-25-50|7,224,kdiba,vvp01,one,2006-4-10_12-25-50,235.722217,143.149870,SHARED,0.712399,1.658397,...,-96.429528,0.000000,kdiba|vvp01|one|2006-4-10_12-25-50,2006-04-10 12:25:50,2,1,False,True,True,True
kdiba|vvp01|two|2006-4-10_12-58-3|37,327,kdiba,vvp01,two,2006-4-10_12-58-3,38.905182,137.605942,SHARED,0.645332,0.570190,...,NaN,-25.589086,kdiba|vvp01|two|2006-4-10_12-58-3,2006-04-10 12:58:03,3,1,False,True,True,True


In [15]:
neuron_replay_stats_df

,Unnamed: 0,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,...,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,has_long_pf,has_short_pf
neuron_uid,,,,,,,,,,,,,,,,,,,,,
kdiba|gor01|one|2006-6-12_15-55-31|2,0,kdiba,gor01,one,2006-6-12_15-55-31,184.118896,NaN,LEFT_ONLY,1.219400,0.702153,...,NaN,-7.638520,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,False
kdiba|gor01|one|2006-6-12_15-55-31|3,1,kdiba,gor01,one,2006-6-12_15-55-31,100.095178,NaN,LEFT_ONLY,0.736015,0.560419,...,NaN,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,False
kdiba|gor01|one|2006-6-12_15-55-31|4,2,kdiba,gor01,one,2006-6-12_15-55-31,157.384077,149.745557,SHARED,0.627405,0.389968,...,197.486305,7.638520,141.312616,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,True
kdiba|gor01|one|2006-6-12_15-55-31|5,3,kdiba,gor01,one,2006-6-12_15-55-31,180.299636,199.395935,SHARED,0.979698,1.661452,...,79.089249,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,True
kdiba|gor01|one|2006-6-12_15-55-31|6,4,kdiba,gor01,one,2006-6-12_15-55-31,NaN,NaN,SHARED,10.865067,13.557341,...,NaN,NaN,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kdiba|pin01|one|fet11-01_12-58-54|28,393,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,1.316956,...,100.438316,74.542266,40.998246,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,0,0,True,True,True
kdiba|pin01|one|fet11-01_12-58-54|29,394,kdiba,pin01,one,fet11-01_12-58-54,57.576513,102.301872,SHARED,0.929991,0.838893,...,126.528109,29.816906,11.181340,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,0,0,True,True,True
kdiba|pin01|one|fet11-01_12-58-54|30,395,kdiba,pin01,one,fet11-01_12-58-54,120.937439,128.391665,SHARED,0.952187,0.842384,...,186.161921,-18.635566,-52.179586,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,0,0,True,True,True


In [17]:
# non_disappearing_endcap_cells_df
significant_distant_remapping_endcap_cells_df
# 'track_membership'

,Unnamed: 0,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,...,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,has_long_pf,has_short_pf,has_significant_distance_remapping
neuron_uid,,,,,,,,,,,,,,,,,,,,,
kdiba|gor01|one|2006-6-12_15-55-31|32,30,kdiba,gor01,one,2006-6-12_15-55-31,249.046314,168.841856,SHARED,1.216096,2.123720,...,-15.277040,NaN,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,True,True
kdiba|gor01|one|2006-6-12_15-55-31|45,43,kdiba,gor01,one,2006-6-12_15-55-31,38.987020,92.456658,SHARED,4.865325,2.699642,...,NaN,-7.638520,kdiba|gor01|one|2006-6-12_15-55-31,2006-06-12 15:55:31,8,4,False,True,True,True
kdiba|gor01|two|2006-6-07_16-40-19|64,114,kdiba,gor01,two,2006-6-07_16-40-19,235.301279,103.152020,SHARED,2.832245,5.618467,...,NaN,-15.102773,kdiba|gor01|two|2006-6-07_16-40-19,2006-06-07 16:40:19,1,0,True,True,True,True
kdiba|gor01|two|2006-6-12_16-53-46|24,138,kdiba,gor01,two,2006-6-12_16-53-46,54.226198,179.984124,SHARED,0.202350,2.313323,...,-99.082002,NaN,kdiba|gor01|two|2006-6-12_16-53-46,2006-06-12 16:53:46,9,4,False,True,True,True
kdiba|vvp01|one|2006-4-09_17-29-30|22,195,kdiba,vvp01,one,2006-4-09_17-29-30,34.270932,91.548868,SHARED,0.953844,0.869835,...,0.000000,NaN,kdiba|vvp01|one|2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,True,True,True,True
kdiba|vvp01|one|2006-4-09_17-29-30|42,215,kdiba,vvp01,one,2006-4-09_17-29-30,244.290031,164.100921,SHARED,1.102559,1.466096,...,NaN,NaN,kdiba|vvp01|one|2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,True,True,True,True
kdiba|vvp01|one|2006-4-09_17-29-30|43,216,kdiba,vvp01,one,2006-4-09_17-29-30,30.452403,87.730339,SHARED,1.953906,2.713736,...,NaN,3.818529,kdiba|vvp01|one|2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,True,True,True,True
kdiba|vvp01|one|2006-4-10_12-25-50|7,224,kdiba,vvp01,one,2006-4-10_12-25-50,235.722217,143.149870,SHARED,0.712399,1.658397,...,-96.429528,0.000000,kdiba|vvp01|one|2006-4-10_12-25-50,2006-04-10 12:25:50,2,1,False,True,True,True
kdiba|vvp01|two|2006-4-10_12-58-3|37,327,kdiba,vvp01,two,2006-4-10_12-58-3,38.905182,137.605942,SHARED,0.645332,0.570190,...,NaN,-25.589086,kdiba|vvp01|two|2006-4-10_12-58-3,2006-04-10 12:58:03,3,1,False,True,True,True


## Required for Interactive Plotting and figure displace

In [19]:
neuron_replay_stats_table.index.unique() #['neuron_uid'].unique()

Index(['kdiba|gor01|one|2006-6-12_15-55-31|2',
       'kdiba|gor01|one|2006-6-12_15-55-31|3',
       'kdiba|gor01|one|2006-6-12_15-55-31|4',
       'kdiba|gor01|one|2006-6-12_15-55-31|5',
       'kdiba|gor01|one|2006-6-12_15-55-31|6',
       'kdiba|gor01|one|2006-6-12_15-55-31|7',
       'kdiba|gor01|one|2006-6-12_15-55-31|8',
       'kdiba|gor01|one|2006-6-12_15-55-31|9',
       'kdiba|gor01|one|2006-6-12_15-55-31|10',
       'kdiba|gor01|one|2006-6-12_15-55-31|11',
       ...
       'kdiba|pin01|one|fet11-01_12-58-54|23',
       'kdiba|pin01|one|fet11-01_12-58-54|24',
       'kdiba|pin01|one|fet11-01_12-58-54|25',
       'kdiba|pin01|one|fet11-01_12-58-54|26',
       'kdiba|pin01|one|fet11-01_12-58-54|27',
       'kdiba|pin01|one|fet11-01_12-58-54|28',
       'kdiba|pin01|one|fet11-01_12-58-54|29',
       'kdiba|pin01|one|fet11-01_12-58-54|30',
       'kdiba|pin01|one|fet11-01_12-58-54|31',
       'kdiba|pin01|one|fet11-01_12-58-54|32'],
      dtype='object', name='neuron_uid', lengt

In [22]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
num_sessions: int = 13
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

num_above_diagonal/total_num_points: 194/398
percent_below_diagonal: 50.25125628140703%
percent_above_diagonal: 48.743718592964825%
register_output_file(output_path: /home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-10-22/kdiba/{}_13_across_sessions_firing_rate_index.png, ...)
	 saved "file:///home/halechr/repos/Spike3D/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-10-22/kdiba/%7B%7D_13_across_sessions_firing_rate_index.png"


## Extract output files from all completed sessions:

# 2023-10-06 - `joined_neruon_fri_df` loading

# 2023-10-04 - Load Saved across-sessions-data, process, and produce figures

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

# 'collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl'
across_session_result_long_short_recomputed_inst_firing_rate_file = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
across_session_result_long_short_recomputed_inst_firing_rate_file.exists()

'across_session_result_long_short_inst_firing_rate_2024-06-11_GL.pkl'

In [23]:
# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## INPUTS: BATCH_DATE_TO_USE, BATCH_DATE_TO_USE
print(f"BATCH_DATE_TO_USE: {BATCH_DATE_TO_USE}")
## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

across_session_result_long_short_recomputed_inst_firing_rate_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl"

# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)

AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=collected_outputs_path.resolve(),
                                                 inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)


# "/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl"

# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

BATCH_DATE_TO_USE: 2024-10-22_GL
global_batch_result_inst_fr_file_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl
Saving (file mode 'w+b') pickle file results : "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl"... saved pickle file


In [24]:
# output_path_suffix: str = '2024-09-11_GL'
# output_path_suffix: str = '2024-09-26_GL'
output_path_suffix: str = BATCH_DATE_TO_USE
# graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=global_data_root_parent_path, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, plotting_enabled=True)
graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix, plotting_enabled=True)


global_batch_result_inst_fr_file_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl
Computing loaded session pickle file results : "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl"... done.
num_sessions: 1


AssertionError: expected values to be tuples Tuple[Context, InstantaneousSpikeRateGroupsComputation, inst_fr_t_bin_size] but were: [{IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: (IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>, InstantaneousSpikeRateGroupsComputation(instantaneous_time_bin_size_seconds=1000.0, active_identifying_session_ctx=IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>, LxC_aclus=[109], SxC_aclus=[23, 4, 58], Fig2_Replay_FR=[SingleBarResult(mean=5.067567567567568e-05, std=0.0, values=array([5.06756757e-05]), LxC_aclus=[109], SxC_aclus=[23, 4, 58], LxC_scatter_props=None, SxC_scatter_props=None), SingleBarResult(mean=2.857142857142857e-05, std=0.0, values=array([2.85714286e-05]), LxC_aclus=[109], SxC_aclus=[23, 4, 58], LxC_scatter_props=None, SxC_scatter_props=None), SingleBarResult(mean=5.180180180180182e-05, std=3.662940533173558e-05, values=array([7.77027027e-05, 0.00000000e+00, 7.77027027e-05]), LxC_a

In [ ]:
from PendingNotebookCode import build_and_merge_all_sessions_joined_neruon_fri_df

all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE, included_session_contexts=included_session_contexts)

joined_neruon_fri_df: concatenating dataframes from 0


ValueError: No objects to concatenate

In [30]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext

annotation_man = UserAnnotationsManager()

LxC_uids = []
SxC_uids = []

for a_ctxt in included_session_contexts:
	session_uid = a_ctxt.get_description(separator="|", include_property_names=False)
	session_uid
	session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[a_ctxt].get('session_cell_exclusivity', None)
	LxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.LxC])
	SxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.SxC])
	
# [a_ctxt.get_description(separator="|", include_property_names=False) for a_ctxt in included_session_contexts]

long_short_fr_indicies_analysis_table['XxC_status'] = 'Shared'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, LxC_uids), 'XxC_status'] = 'LxC'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, SxC_uids), 'XxC_status'] = 'SxC'

long_short_fr_indicies_analysis_table
## 2023-10-11 - Get the long peak location

long_short_fr_indicies_analysis_table['long_pf_peak_x'] = neuron_replay_stats_table['long_pf_peak_x']
long_short_fr_indicies_analysis_table

## SAVE:
# long_short_fr_indicies_analysis_table_filename = 'output/2023-10-07_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table_filename: str = f'output/{BATCH_DATE_TO_USE}_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table.to_csv(long_short_fr_indicies_analysis_table_filename)
print(f'saved: {long_short_fr_indicies_analysis_table_filename}')

saved: output/2024-10-22_GL_long_short_fr_indicies_analysis_table.csv


# Loading

In [ ]:
## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-04-GL-Recomp.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-07.pkl'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_load_filepath: Path = Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl').resolve()

# "/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl"
assert inst_fr_output_load_filepath.exists()
inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
 
## Load all across-session tables from the pickles:
# output_path_suffix: str = f'2023-10-07'
# output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
# output_path_suffix: str = '2024-06-12_GL'
# output_path_suffix: str = '2024-09-03_GL'
output_path_suffix: str = '2024-09-26_GL'
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix)
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')

### 2023-11-15 - For manual testing

In [ ]:
# ## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


In [26]:
across_sessions_instantaneous_frs_list

NameError: name 'across_sessions_instantaneous_frs_list' is not defined

# 2024-06-11 - Across Session Outputs

In [29]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='x_frs_index', title='Pf Peak position vs. LapsFRI', ylabel='Lap FRI')
long_short_fr_indicies_analysis_table.plot.scatter(x='long_pf_peak_x', y='y_frs_index', title='Pf Peak position vs. ReplayFRI', ylabel='Replay FRI')

<Axes: title={'center': 'Pf Peak position vs. ReplayFRI'}, xlabel='long_pf_peak_x', ylabel='Replay FRI'>

# 2023-10-10 - Statistics for `across_sessions_bar_graphs`, analysing `across_session_inst_fr_computation` 

In [31]:
import scipy.stats as stats
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_perform_diagonal_line_binomial_test, pho_stats_bar_graph_t_tests

binom_test_chance_result = pho_stats_perform_diagonal_line_binomial_test(long_short_fr_indicies_analysis_table)
print(f'binom_test_chance_result: {binom_test_chance_result}')

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)

# n_total: 823, n_above_diagonal: 457, n_exact_on_diagonal: 0, n_below_diagonal: 366
# binom_test_chance_result: BinomTestResult(k=457, n=823, alternative='two-sided', statistic=0.5552855407047388, pvalue=0.0016893424059938723)
# LxC_Laps_T_result: TtestResult(statistic=12.249237714915296, pvalue=3.8446809431691085e-08, df=12)
# SxC_Laps_T_result: TtestResult(statistic=-12.413163641851535, pvalue=5.768107640498047e-07, df=9)
# LxC_Replay_T_result: TtestResult(statistic=-0.7636543941504783, pvalue=0.4598247399068105, df=12)
# SxC_Replay_T_result: TtestResult(statistic=-3.069767144140489, pvalue=0.01335902191105584, df=9)


n_total: 394, n_above_diagonal: 200, n_exact_on_diagonal: 0, n_below_diagonal: 194
binom_test_chance_result: BinomTestResult(k=200, n=394, alternative='two-sided', statistic=0.5076142131979695, pvalue=0.8011598917108477)


NameError: name 'across_session_inst_fr_computation' is not defined

In [ ]:
debug_print: bool = False
enable_neptune: bool = True

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"

	
    # notebook_filepath

	# '/home/halechr/repos/Spike3D/BatchInteractiveProcessing_2024-06-11_GL.ipynb'

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        # neptuner.run = neptune.init_run(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath]) # see git_ref=GitRef(repository_path="/path/to/repo")
        # Add the session_context properties to the run: {'format_name': 'kdiba', 'animal': 'vvp01', 'exper_name': 'two', 'session_name': '2006-4-09_16-40-54'}

        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])

        # Create an AutoValueConvertingNeptuneRun and copy the attributes
        # neptuner.run = AutoValueConvertingNeptuneRun(base_run._client, base_run._uuid, base_run._url)
    
        params = {'BATCH_DATE_TO_USE': BATCH_DATE_TO_USE, "run_workstation": "GL"}
        neptuner.run["parameters"] = params

        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None

In [ ]:
for a_save_path, a_save_dict in registered_output_files.items():
	a_save_dict['fig']
	a_save_dict['context']
	neptuner.figures.upload(File(a_save_path.resolve().as_posix()))

In [ ]:

neptuner_run[f'output_files/across_sessions_fig_2'] = _out_fig_2

In [ ]:
if (neptuner is not None) and (neptuner_run is not None):
    neptuner.stop()

# KDiba Session Discovery - Determines contexts to process - 2024-09-19

In [ ]:
# from pathlib import Path
# from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
# from neuropy.utils.result_context import print_identifying_context_array_code

# known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
# global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
# assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
sessions_df, export_folder_path = KDibaOldDataSessionFormatRegisteredClass.find_build_and_save_sessions_experiment_datetime_df_csv(
	                                                                        # global_data_root_parent_path=global_data_root_parent_path,
                                                                            export_csv_path=Path('EXTERNAL/PhoDibaPaper2024Book/data/sessions_experiment_datetime_df.csv').resolve(),
																			bad_sessions_csv_path=Path('EXTERNAL/PhoDibaPaper2024Book/data/2024-09-23_bad_sessions_table.csv').resolve(),
)
sessions_df = sessions_df.reset_index(drop=True, inplace=False)
sessions_df


In [ ]:
# old_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), #
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
#     # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), #
# ]
    
# new_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_14-59-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-27_18-21-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_12-17-27'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_11-0-53'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_12-35-59'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-2-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-55-7'),
# ]

# completely_new_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_11-0-53'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-27_18-21-57'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-2-0'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-55-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_14-59-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_12-17-27'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_12-35-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3'),
# ]


In [ ]:
completely_new_included_session_contexts = set(new_included_session_contexts) - set(old_included_session_contexts)
print_identifying_context_array_code(completely_new_included_session_contexts, array_name='completely_new_included_session_contexts')